In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, MaxPooling2D, Flatten, Dense

In [2]:
def bottleneck_residual_block(X, kernel_size, filters, reduce=False, s=2):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    if reduce:
        X_shortcut = Conv2D(filters = F3, kernel_size=(1,1), strides = (s, s)) (X_shortcut)
        X_shortcut = BatchNormalization(axis=3)(X_shortcut)
        
        # sets the strides of the first convolutional layer to be similar to the shortcut strides
        X = Conv2D(filters = F1, kernel_size=(1,1), strides=(s, s), padding='valid')(X)
        X = BatchNormalization(axis=3)(X)
        X = Activation('relu')(X)
    
    else:
        # First component of main path
        X = Conv2D(filters = F1, kernel_size=(1,1), strides=(1, 1), padding='valid')(X)
        X = BatchNormalization(axis=3)(X)
        X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size=kernel_size, strides=(1, 1), padding='same')()
    X = BatchNormalization(axis=3) (X)
    X = Activation('relu')(X)
    
    # Third component of main path
    X = Conv2D(filters = F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')
    X = BatchNormalization(axis=3) (X)
    X = Activation('relu')(X)
    
    # Final Step
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

### BatchNormalization(axis=3)
  - axis=3 파라미터는 채널 차원이 입력의 마지막 차원에 위치할 때 사용
  - 이미지 데이터의 일반적인 형태는 (배치 크기, 높이, 너비, 채널 수):
    - axis=3은 이 채널 차원을 정규화 대상으로 지정

In [3]:
def ResNet50(input_shape, classes):
    X_input = Input(input_shape)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X_input)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = bottleneck_residual_block(X, 3, [64, 64, 256], reduce=True, s=1)
    X = bottleneck_residual_block(X, 3, [64, 64, 256])
    X = bottleneck_residual_block(X, 3, [64, 64, 256])

    # Stage 3
    X = bottleneck_residual_block(X, 3, [128, 128, 512], reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [128, 128, 512])
    X = bottleneck_residual_block(X, 3, [128, 128, 512])
    X = bottleneck_residual_block(X, 3, [128, 128, 512])

    # Stage 4
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = bottleneck_residual_block(X, 3, [512, 512, 2048], reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [512, 512, 2048])
    X = bottleneck_residual_block(X, 3, [512, 512, 2048])

    # AVGPOOL
    X = AveragePooling2D((1,1))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [5]:
input_shape = (224, 224, 3)
classes = 10

model = ResNet50(input_shape=input_shape, classes=classes)

model.summary()

ValueError: The first argument to `Layer.call` must always be passed.

In [ ]:
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers
import numpy as np

epochs = 200
batch_size = 256

reduce_lr= ReduceLROnPlateau(
    monitor='val_loss',
    factor=np.sqrt(0.1),
    patience=5,
    min_lr=0.5e-6
)

SGD = optimizers.SGD()

model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD,
    metrics=['accuracy']
)

model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs, callbacks=[reduce_lr])